In [50]:
import numpy as np

def random_tt_binary_fixed(seed=None):
    """
    Create a random TT with 7 binary modes and ranks [1,2,4,8,8,4,2,1].

    Parameters
    ----------
    seed : int or None
        Random seed for reproducibility.

    Returns
    -------
    cores : list of ndarray
        List of 7 cores, shapes as described.
    """
    if seed is not None:
        np.random.seed(seed)
    dims  = [2] * 7
    ranks = [1, 2, 4, 8, 8, 4, 2, 1]
    cores = []
    for k in range(7):
        rL, n, rR = ranks[k], dims[k], ranks[k+1]
        G = np.random.randn(rL, n, rR)
        cores.append(G)
    return cores

def tt_to_tensor(cores):
    """
    Reconstruct the full tensor from its TT cores.
    """
    # start with first core, squeeze r0=1
    A = np.squeeze(cores[0], axis=0)   # shape (2, r1)
    for G in cores[1:]:
        A = np.tensordot(A, G, axes=([-1],[0]))
    # squeeze final r7=1
    return np.squeeze(A, axis=-1)      # final shape (2,2,...,2), 7 modes

if __name__ == "__main__":
    cores = random_tt_binary_fixed(seed=2025)
    print("Core shapes:", [c.shape for c in cores])
    A = tt_to_tensor(cores)
    print("Reconstructed tensor shape:", A.shape)
    # Now A[i1,i2,...,i7] exists for each ik∈{0,1}

print(type(cores))
print(type(cores[0]))

Core shapes: [(1, 2, 2), (2, 2, 4), (4, 2, 8), (8, 2, 8), (8, 2, 4), (4, 2, 2), (2, 2, 1)]
Reconstructed tensor shape: (2, 2, 2, 2, 2, 2, 2)
<class 'list'>
<class 'numpy.ndarray'>


In [51]:
W = []
nk = []
for k in range (len(cores)):
     nk.append(len(cores[k-1][0,0,:]))
print(nk)

for k in range(len(cores)):
    # reshape & SVD 
    j, i, n_k = cores[k].shape
    
    cores[k] = np.reshape(cores[k], (j * i, n_k))

    #print(cores[k].shape)
    U, S, V = np.linalg.svd(cores[k])
    S_prime = np.zeros_like(cores[k])
    
    #print(S_prime.shape)
    R = np.diag(S) @ V
    if k != len(cores) - 1:
        cores[k+1] = np.einsum('j r,r i k->j i k', R, cores[k+1])

    # calcolo dei qubit coinvolti
    start = k + 1
    mn    = min(start, int(np.log2(nk[k])))
    diff  = int(start - mn)
    qubits = list(range(diff-1, start))
    print(U.shape)
    U_list = U.tolist()
    W.append([U_list, qubits])

# Stampa di controllo
for idx, (U_list, qubits) in enumerate(W):
    print(f"Unitary {idx}: acts on qubits {qubits}, matrix with dimension {len(U_list)}")


[1, 2, 4, 8, 8, 4, 2]
(2, 2)
(4, 4)
(8, 8)
(16, 16)
(16, 16)
(8, 8)
(4, 4)
Unitary 0: acts on qubits [0], matrix with dimension 2
Unitary 1: acts on qubits [0, 1], matrix with dimension 4
Unitary 2: acts on qubits [0, 1, 2], matrix with dimension 8
Unitary 3: acts on qubits [0, 1, 2, 3], matrix with dimension 16
Unitary 4: acts on qubits [1, 2, 3, 4], matrix with dimension 16
Unitary 5: acts on qubits [3, 4, 5], matrix with dimension 8
Unitary 6: acts on qubits [5, 6], matrix with dimension 4
